In [1]:
import numpy as np
import pickle
import cv2
import os
import keras
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras import backend as K
import pickle
import joblib
from keras.models import model_from_json
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization

from keras.applications import ResNet50
from keras.optimizers import Adam

import efficientnet.keras as efn 
import tensorflow as tf
from tensorflow.keras.callbacks import Callback
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import Adadelta


In [2]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = "true"


1 Physical GPUs, 1 Logical GPUs


In [3]:
ph1=r"C:\Users\aanis\Desktop\plantvillage1\aa\segmented\tomato"###TOMATO SEGMENTED 
li=0
x=[]
y=[]
classes=[]
for i in os.listdir(ph1):
    lab=[0,0,0,0,0,0,0,0,0,0]
    lab[li]=1
    ph2=ph1+f'\{i}'
    #print(ph2)
    li+=1
    classes.append(i)
    for j in os.listdir(ph2):
        #print(j)
        ph3=ph2+f'\{j}'
        #print(x,np.array(cv2.imread(ph3)).reshape(256*256*3))
        #print(lab)
        x.append(np.array(cv2.imread(ph3)))
        y.append(lab)
        

In [4]:
print(np.array(y).shape)
x_train, x_test, y_train, y_test = train_test_split(np.array(x), np.array(y), test_size=0.3, random_state = 42) 
del x
del y
print(type(x_train))

(18159, 10)
<class 'numpy.ndarray'>


In [5]:
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,horizontal_flip=True, fill_mode="nearest")

In [6]:
#wpath=r"C:\Users\aanis\Downloads\resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"

#resnet = ResNet50(weights=wpath, include_top=False, input_shape=(256, 256, 3))
#x_train=np.array(x_train.astype("f"))
#x_test=np.array(x_test.astype("f"))

#tf.cast(x_train, tf.float32)
#tf.cast(y_train, tf.float32)
#tf.cast(x_test, tf.float32)
#tf.cast(y_test, tf.float32)

In [6]:
import efficientnet.keras as efn 
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import Callback
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import Adadelta

checkpoint_path="train_ckpt/cp.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_best_only=True)


# Model
## Define the base model with EfficientNet weights
wpath=r"C:\Users\aanis\Downloads\efficientnet-b0_weights_tf_dim_ordering_tf_kernels_autoaugment_notop.h5"
model = efn.EfficientNetB0(weights = wpath, 
                           include_top = False, 
                           input_shape = (256, 256, 3))

## Output layer
x = model.output
tf.cast(x, tf.float32)

x = GlobalAveragePooling2D()(x)

predictions = Dense(10, activation="softmax")(x)

## Compile and run
model = Model(inputs=model.input, outputs=predictions)

model.compile(optimizer='adam',
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()
BS=8
model_history = model.fit(aug.flow(x_train, y_train, batch_size=BS),
                            validation_data=(x_test, y_test),
                            steps_per_epoch=len(x_train) // BS,
                            validation_steps=len(x_test) //BS,
                            epochs=15,
                            verbose=1,
                            callbacks=[cp_callback])

C:\Users\aanis\anaconda3\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:38: UserWarning: You are currently using a nightly version of TensorFlow (2.4.0-dev20201023). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 128, 128, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 128, 128, 32) 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 128, 128, 32) 0           stem_bn[0][0]                    
______________________________________________________________________________________________

Epoch 1/15
1588/1588 [==============================] - 1999s 1s/step - loss: 0.7252 - accuracy: 0.7668 - val_loss: 0.3832 - val_accuracy: 0.8741

Epoch 00001: val_loss improved from inf to 0.38316, saving model to train_ckpt\cp.ckpt
Epoch 2/15
1588/1588 [==============================] - 1973s 1s/step - loss: 0.2916 - accuracy: 0.8990 - val_loss: 0.2785 - val_accuracy: 0.9121

Epoch 00002: val_loss improved from 0.38316 to 0.27850, saving model to train_ckpt\cp.ckpt
Epoch 3/15
1588/1588 [==============================] - 1971s 1s/step - loss: 0.2071 - accuracy: 0.9320 - val_loss: 0.1753 - val_accuracy: 0.9438

Epoch 00003: val_loss improved from 0.27850 to 0.17534, saving model to train_ckpt\cp.ckpt
Epoch 4/15
1588/1588 [==============================] - 1972s 1s/step - loss: 0.1967 - accuracy: 0.9328 - val_loss: 0.2677 - val_accuracy: 0.9220

Epoch 00004: val_loss did not improve from 0.17534
Epoch 5/15
1588/1588 [==============================] - 1971s 1s/step - loss: 0.1513 - accur

In [7]:
model.save("effecientnettest2")

C:\Users\aanis\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
C:\Users\aanis\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\base_layer.py:1397: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.
INFO:tensorflow:Assets written to: effecientnettest2\assets


In [10]:
#train=reconstructed_model
train=model
acc = train.History['accuracy']
val_acc = train.history['val_accuracy']
loss = train.history['loss']
val_loss = train.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

NameError: name 'model' is not defined

In [5]:
reconstructed_model = keras.models.load_model("effecientnettest1")
scores = reconstructed_model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")


171/171 [==============================] - 168s 945ms/step - loss: 0.0920 - accuracy: 0.9725
Test Accuracy: 97.24669456481934


In [6]:
y_pred=reconstructed_model.predict(x_test)

In [7]:
y_test=np.argmax(y_test, axis=1)
y_pred=np.argmax(y_pred, axis=1)

In [8]:
print(classification_report(y_test, y_pred,target_names=classes))

                                               precision    recall  f1-score   support

                      Tomato___Bacterial_spot       0.94      0.98      0.96       626
                        Tomato___Early_blight       0.96      0.93      0.95       307
                             Tomato___healthy       0.96      1.00      0.98       459
                         Tomato___Late_blight       0.97      0.97      0.97       583
                           Tomato___Leaf_Mold       0.99      0.93      0.96       283
                  Tomato___Septoria_leaf_spot       0.96      0.99      0.97       529
Tomato___Spider_mites Two-spotted_spider_mite       0.96      0.99      0.97       532
                         Tomato___Target_Spot       0.97      0.94      0.95       420
                 Tomato___Tomato_mosaic_virus       0.95      0.99      0.97       118
       Tomato___Tomato_Yellow_Leaf_Curl_Virus       1.00      0.98      0.99      1591

                                     accu

In [11]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, y_pred))

[[ 616    0    0    1    0    2    2    5    0    0]
 [   0  287    2    9    0    6    0    1    2    0]
 [   0    0  459    0    0    0    0    0    0    0]
 [   0    9    3  563    3    3    2    0    0    0]
 [   2    1    2    0  263    2   12    0    1    0]
 [   4    0    0    0    0  522    2    1    0    0]
 [   0    1    0    0    0    0  525    6    0    0]
 [   3    1    9    1    0    8    5  393    0    0]
 [   0    0    0    0    0    0    1    0  117    0]
 [  27    1    2    5    0    0    0    0    3 1553]]


In [ ]:
filepath="train_ckpt/cp.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5)

In [ ]:
model_history = model.fit(aug.flow(x_train, y_train, batch_size=BS),
                            validation_data=(x_test, y_test),
                            steps_per_epoch=len(x_train) // BS,
                            validation_steps=len(x_test) //BS,
                            epochs=10,
                            verbose=1,
                            callbacks=[cp_callback])
